# Intro
**ModelTrainer** trains all the models found in the *model_type*'s json file (i.e. *cifar.json*)

In [7]:
import os, pdb, json, re, vis_utils
# Import codes to obtain requisite commands
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential, load_model, Model, model_from_json
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.utils import np_utils, generic_utils
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

# File Loaders
1. **model_json**: This contains model details. The format involves a model set (i.e. *cifar*, etc) and relevant details such as image dimensions, training data size, etc

2. **top_level**: Lists the top level files (at this time, they are <span style="color:#006400">['jsons', 'activations', 'data', 'max_activations','history', 'tensorlogs', 'weights']</span>)

3. **dir_paths**: The relative directory paths to the **top_level** directories, i.e. *cifar/activations*

4. **jsons** : The list of *model.json* files, i.e. *cifar.json* within the **cifar/json** folder

5. **model_dirs**: The directory that each of the *model.json*'s training and activation data should reside, i.e. **weights/cifar_1/**

In [8]:
model_json = vis_utils.json_load('models.json')

In [9]:
model_type = 'cifar'
top_level= [_dir for _dir in os.listdir(model_type) if not _dir.startswith('.')]
dir_paths = {entry:os.path.join(model_type, entry) for entry in top_level}
jsons = os.listdir(dir_paths['jsons'])
model_dirs = [os.path.splitext(_item)[0] for _item in jsons]
train_data_dir = 'data/' + model_type + '/train'
test_data_dir = 'data/' + model_type + '/test'
class_name = {i:str(model_json[model_type]['labels'][i]) for i in range(len(model_json[model_type]['labels']))}

# Keras Image Data generator
Generate the batched keras image data files through **ImageDataGenerator**. Sets up the training and test data, using information from **model_json**

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)
batch_size = model_json[model_type]['batch']
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(model_json[model_type]['size']['width'],model_json[model_type]['size']['height']),
        batch_size=batch_size,
        classes=[str(_item) for _item in model_json[model_type]['labels']],
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(model_json[model_type]['size']['width'],model_json[model_type]['size']['height']),
        batch_size=batch_size,
        classes=[str(_item) for _item in model_json[model_type]['labels']],
        class_mode='categorical')

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


# <span style="color:red">Training</span>
Trains every model stored in *model.json* i.e. *cifar.json* - *cifar_1.json*, *cifar_2.json*, etc. The relevant data are stored in:
1. **history**: The model histories (Keras based) are stored in *model_type/history/model.csv* aka **cifar/history/cifar_1.csv**
2. **Tensorboard**: The Tensorboard data is stored in **cifar/tensorlogs/cifar_1/**
3. **Epoch Weights**: The epoch weights are stored in **cifar/weights/cifar_1/** as *weights-#.hdf5*
3. **Best Model**: The best model's weights are stored in **cifar/weights/cifar_1/** as *best.hdf5*

This sets up what is completed, so we can skip it in training

In [13]:
if os.path.exists(os.path.join(dir_paths['supplement'], 'completed.txt')):
    _cont = open(os.path.join(dir_paths['supplement'], 'completed.txt'), 'r')
    dat = _cont.read().split()
    _cont.close()
else:
    dat = []
for idx in range(len(jsons)):
    # Setting up the directory structures
    if jsons[idx] in dat:
        continue
    print('Starting with ' + jsons[idx])
    dir_list = vis_utils.dir_list_returner(dir_paths, ['weights','tensorlogs','history','supplement'], model_dirs[idx])
    #Set up supplemental parameters
    compile_params = vis_utils.json_load(os.path.join(dir_list['supplement'], 'compile.json'))
    # Loading the Model file
    with open(os.path.join(dir_paths['jsons'],jsons[idx]), 'r') as model_file:
        model = model_from_json(model_file.read())
    model.compile(loss = compile_params['loss'], optimizer = compile_params['optimizer'], metrics=compile_params['metrics'])
    # Setting up the callbacks
    tensorboard_callback = TensorBoard(log_dir=dir_list['tensorlogs']+'/', histogram_freq=0, write_graph=True, write_images=False)
    best_callback = ModelCheckpoint(os.path.join(dir_list['weights'],'best.hdf5'), monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    checkpoint_callback = ModelCheckpoint(os.path.join(dir_list['weights'],'weights-{epoch:02d}.hdf5'), monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    #------------------------------------------------------------------------------------------------
    # Train
    model_hist = model.fit_generator(
            train_generator,
            steps_per_epoch=model_json[model_type]['train']/model_json[model_type]['batch'],
            epochs = model_json[model_type]['epochs'],
            validation_data = test_generator,
            validation_steps = model_json[model_type]['test']/model_json[model_type]['batch'],
            verbose = 1,
            callbacks=[tensorboard_callback, checkpoint_callback, best_callback])
    with open(os.path.join(dir_paths['history'], jsons[idx]), 'w') as _compile:
            json.dump(model_hist.history, _compile)
    print('Completed with ' + jsons[idx])
    with open(os.path.join(dir_paths['supplement'], 'completed.txt'), 'a') as _compile:
            _compile.write(jsons[idx]+'\n')

Starting with cifar_6.json
Epoch 1/15
781/781 [==============================] - 67s - loss: 1.6792 - acc: 0.3777 - val_loss: 1.3673 - val_acc: 0.4955
Epoch 2/15
781/781 [==============================] - 66s - loss: 1.2690 - acc: 0.5434 - val_loss: 1.1498 - val_acc: 0.5864
Epoch 3/15
781/781 [==============================] - 66s - loss: 1.0633 - acc: 0.6209 - val_loss: 1.0273 - val_acc: 0.6359
Epoch 4/15
781/781 [==============================] - 66s - loss: 0.8959 - acc: 0.6802 - val_loss: 0.9419 - val_acc: 0.6614
Epoch 5/15
781/781 [==============================] - 66s - loss: 0.7670 - acc: 0.7281 - val_loss: 0.9374 - val_acc: 0.6749
Epoch 6/15
781/781 [==============================] - 66s - loss: 0.6241 - acc: 0.7794 - val_loss: 0.9774 - val_acc: 0.6683
Epoch 7/15
781/781 [==============================] - 66s - loss: 0.5115 - acc: 0.8199 - val_loss: 1.0137 - val_acc: 0.6803
Epoch 8/15
781/781 [==============================] - 66s - loss: 0.4181 - acc: 0.8532 - val_loss: 1.1594